In [31]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook as tqdm

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torchtext.data import Field, TabularDataset, BucketIterator, Iterator
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import AutoModel, BertTokenizerFast, AdamW
from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

import torch.optim as optim

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import plotly.express as px
import seaborn as sns

import time
import warnings
import random
%matplotlib inline
warnings.filterwarnings('ignore')

In [16]:
path = 'model_save/'

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [66]:
gpt_configuration = GPT2Config.from_pretrained(f'{path}config.json', output_hidden_states=False)
gpt_model = GPT2LMHeadModel.from_pretrained(f"{path}pytorch_model.bin", config=configuration)

In [67]:
gpt_tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path=f'{path}',
                                          vocab_file=f'{path}vocab.json', merges_file=f'{path}merges.txt',
                                          bos_token='<|sot|>', eos_token='<|eot|>', pad_token='<|pad|>')

In [68]:
gpt_tokenizer

PreTrainedTokenizer(name_or_path='model_save/', vocab_size=50257, model_max_len=1024, is_fast=False, padding_side='right', special_tokens={'bos_token': AddedToken("<|sot|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|eot|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<|pad|>'})

In [69]:
gpt_model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [2]:
class DetectiongGAN(torch.NN):
    def __init__(self, dataframe, num_labels=2, GPT2model=gpt_model,
                 gpt_tokenizer=gpt_tokenizer, BERTmodel=, BERTtokenizer=)
        self.dataframe = dataframe

        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

        self.bert_tokenizer = BertTokenizer.from_pretrained(bert_pretrained_model)
        self.discriminator = BertForSequenceClassification.from_pretrained(bert_pretrained_model, num_labels = num_labels).to(self.device)
        self.bert_optimizer = BertAdam(self.discriminator.parameters(), lr = 0.00005, warmup = 0.1, t_total = 1000)

        self.gpt2_tokenizer = GPT2Tokenizer.from_pretrained(gpt_pretrained_model)
        self.generator = GPT2model
        self.gpt2_optimizer = OpenAIAdam(self.generator.parameters(), lr = 0.0001, warmup = 0.1, t_total = 1000

SyntaxError: invalid syntax (<ipython-input-2-e59a0c31b8e8>, line 1)